<a href="https://colab.research.google.com/github/saishshinde15/CrewAI/blob/main/Linkedin_Blog_creator_using_CrewAi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 219.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.7/160.7 kB 6.8 MB/s e

In [43]:
pip install langchain langchain-community

  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
Using cached langchain_core-0.1.53-py3-none-any.whl (303 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-groq 0.2.1 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 0.1.53 which is incompatible.


In [44]:
import os

In [45]:
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "api_key"
llm = ChatCohere()


In [46]:
from crewai import Agent, Task, Crew

In [47]:
linkedin_post_planner = Agent(
    role="LinkedIn Post Planner",
    goal="Craft engaging and insightful LinkedIn posts on {topic} to drive engagement and foster meaningful professional discussions.",
    backstory="You're responsible for planning LinkedIn posts about {topic}. "
              "Your goal is to gather valuable insights and data that resonate with a professional audience, "
              "helping them gain knowledge and make well-informed decisions. "
              "Your content serves as a foundation for creating posts that spark conversations and build thought leadership.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [48]:
linkedin_writer = Agent(
    role="LinkedIn Content Writer",
    goal="Craft engaging and insightful LinkedIn posts on the topic: {topic}, tailored to a professional audience.",
    backstory="You are responsible for writing LinkedIn posts about {topic}. "
              "Your work is guided by the LinkedIn Post Planner, who provides an outline and relevant context. "
              "You follow the outlined objectives to create content that sparks meaningful discussions and professional engagement. "
              "Your posts offer objective insights supported by credible information, while clearly distinguishing between factual statements and personal viewpoints. "
              "The goal is to provide value to readers while fostering a sense of community and thought leadership.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)


In [49]:
linkedin_editor = Agent(
    role="LinkedIn Post Editor",
    goal="Refine LinkedIn posts to align with the professional tone and style of the organization.",
    backstory="You are an editor specializing in LinkedIn content. "
              "Your task is to review and enhance posts drafted by the Content Writer, "
              "ensuring they adhere to professional communication standards, "
              "present balanced perspectives, and engage the audience effectively. "
              "You also ensure posts avoid overly controversial topics while remaining insightful and relevant.",
    allow_delegation=False,
    verbose=True,
    llm=llm

)

In [50]:
linkedin_humanizer = Agent(
    role="LinkedIn Post Humanizer",
    goal="Transform a blog post into a conversational, engaging LinkedIn post that feels authentic and personal, ensuring originality and avoiding plagiarism.",
    backstory="You specialize in adapting blog content for LinkedIn, with a focus on humanizing the tone to resonate with a professional audience. "
              "Your task is to take a structured blog post and reframe it as a natural, conversational LinkedIn post. "
              "You aim to create content that fosters genuine connections, encourages discussions, and maintains a unique voice. "
              "While ensuring factual accuracy, you also add a personal touch, avoiding direct replication of the original text.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)


In [51]:
linkedin_post_plan = Task(
    description=(
        "1. Research the latest trends, thought leaders, and breaking news relevant to {topic}.\n"
        "2. Identify the target professional audience, focusing on their challenges, goals, and areas of interest.\n"
        "3. Create a structured post outline, including a compelling hook, key insights, and an actionable closing statement.\n"
        "4. Incorporate LinkedIn-friendly keywords, hashtags, and data points to maximize reach and engagement."
    ),
    expected_output="A comprehensive LinkedIn post plan document "
        "with a clear outline, audience insights, "
        "engagement strategies, and relevant resources.",
    agent=linkedin_post_planner,
)

In [52]:
linkedin_write = Task(
    description=(
        "1. Use the LinkedIn post plan to craft an engaging and insightful post on {topic}.\n"
        "2. Incorporate LinkedIn-friendly keywords and hashtags naturally.\n"
        "3. Ensure sections and subtitles (if applicable) are concise and designed to capture attention.\n"
        "4. Structure the post with a strong hook, a well-developed body offering actionable insights, and a compelling call-to-action or takeaway.\n"
        "5. Add some emojis if necessary in the later half of the post.Try to use some professional emojis"
        "5. Proofread for grammatical accuracy, readability, and alignment with the organization’s professional tone and voice."
    ),
    expected_output="A polished LinkedIn post in markdown format, "
        "ready for publishing, with clear sections and concise paragraphs "
        "that encourage engagement and conversation.",
    agent=linkedin_writer,
)

In [53]:
linkedin_edit = Task(
    description=(
        "Review the drafted LinkedIn post for grammatical accuracy, "
        "clarity, and alignment with the organization's professional tone and voice. "
        "Ensure the content is engaging and optimized for LinkedIn's audience."
    ),
    expected_output="A refined LinkedIn post in markdown format, "
                    "ready for publishing, with concise paragraphs and a clear, professional tone.",
    agent=linkedin_editor
)

In [54]:
linkedin_humanize = Task(
    description=(
        "Transform the provided blog post into a LinkedIn post with a natural, conversational tone. \n"
        "1. Ensure the content feels authentic and personal, resonating with a professional audience.\n"
        "2. Reframe the content to encourage engagement and spark discussions, while maintaining factual accuracy.\n"
        "3. Avoid direct replication of the blog post to ensure originality and prevent plagiarism.\n"
        "4. Highlight key insights and include relevant hashtags and calls to action for maximum reach."
    ),
    expected_output="An engaging LinkedIn post in markdown format, "
                    "ready for publication, with a conversational tone "
                    "and clear, concise paragraphs that drive professional interaction.",
    agent=linkedin_humanizer
)

In [55]:
crew=Crew(
    agents=[linkedin_post_planner, linkedin_writer, linkedin_editor, linkedin_humanizer],
    tasks=[linkedin_post_plan, linkedin_write, linkedin_edit, linkedin_humanize],
    verbose=2
)

In [56]:
result = crew.kickoff(inputs={"topic": "Transformer models"})

 [DEBUG]: == Working Agent: LinkedIn Post Planner
 [INFO]: == Starting Task: 1. Research the latest trends, thought leaders, and breaking news relevant to Transformer models.
2. Identify the target professional audience, focusing on their challenges, goals, and areas of interest.
3. Create a structured post outline, including a compelling hook, key insights, and an actionable closing statement.
4. Incorporate LinkedIn-friendly keywords, hashtags, and data points to maximize reach and engagement.


> Entering new CrewAgentExecutor chain...
Thought: I'm ready to craft an exceptional LinkedIn post plan on Transformer models, engaging professionals and fostering meaningful connections.

Final Answer: # LinkedIn Post Plan: Unlocking the Power of Transformer Models

## Introduction:

**Hook:** Are you ready to revolutionize your approach to AI and natural language processing? It's time to dive into the world of Transformer models, the cutting-edge technology taking the industry by storm!

**

In [57]:
from IPython.display import Markdown
Markdown(result)

# Unlocking the Power of #TransformerModels

🌟 Are you ready to dive into the world of AI and NLP? 🌟

Today, I want to share my excitement about a technology that has been making waves in the industry: Transformer models! These innovative models have revolutionized the way we approach natural language processing and are set to transform various sectors.

## 📰 A Journey Begins with a Seminal Paper 📰
It all began with a brilliant publication by Vaswani et al., which introduced the concept of 'Attention' and changed the game for sequence transduction tasks. This paper laid the foundation for the incredible advancements we see today! 📝

## 🤗 Hugging Face: Democratizing AI 🤗
Hugging Face, a community favorite, has played a significant role in making Transformer models accessible. Their platform offers a treasure trove of pre-trained models and datasets, empowering developers and researchers to experiment and innovate. 🛠️

## 🩺 Healthcare's New Ally 🩺
One of the most remarkable impacts of Transformers is in healthcare. Imagine a model that can accurately diagnose medical conditions and suggest personalized treatments! A study by Brown et al. showcases how Transformers are saving lives and improving patient care. 🌡️

## 🌱 Evolution and Real-World Success 🌱
The Transformer architecture has evolved, becoming more powerful and versatile. From healthcare to finance and global communication, these models excel at various tasks:
- **Healthcare:** Early disease detection and personalized medicine.
- **Finance:** Securing transactions with fraud detection.
- **Language Translation:** Enabling real-time global conversations.

## 💡 Technical Deep Dive: Self-Attention 💡
The secret sauce of Transformers? The self-attention mechanism! It allows the model to focus on relevant words in a sentence, ensuring contextually rich outputs. This is achieved through the encoder-decoder architecture, a true engineering marvel! 💻

## 🌟 Challenges and the Road Ahead 🌟
As we embrace the potential of Transformers, we must also address challenges. Ethical AI development and bias mitigation are essential. Additionally, ongoing research focuses on improving efficiency and exploring new architectures. 🌅

## 🌈 Let's Chat and Learn Together! 🌈
- Have you worked with Transformer models? What was your experience like?
- How do you see Transformers shaping your industry's future?
- Let's engage in a meaningful discussion! Share your thoughts, insights, and questions in the comments. Let's learn and grow together! 🌱

Remember, the AI and NLP community thrives on knowledge sharing. Stay tuned for more insightful content, and let's keep the conversation going! 📢

#AIInnovation #NLPBreakthroughs #HuggingFaceCommunity #SelfAttentionMechanism #HealthcareTech #DataScience #EthicalAI #AICommunity